In [0]:
import requests
import datetime
import time
import pandas as pd

In [0]:
# Your API key (USE SECRETS IN PRODUCTION!)
API_KEY = "YOUR_API_KEY_HERE"

# Define origins and destinations
origins = ["Kukatpally, Hyderabad", "Miyapur, Hyderabad"]
destinations = ["Gachibowli, Hyderabad", "Banjara Hills, Hyderabad"]

# Define date and time range
start_date = datetime.date(2025, 5, 25)
end_date = datetime.date(2025, 5, 31)
hour_range = range(7, 24)  # 7 AM to 11 PM

# Storage for results
results = []

# Loop through dates and hours
for single_date in (start_date + datetime.timedelta(n) for n in range((end_date - start_date).days + 1)):
    for hour in hour_range:
        departure_time = int(datetime.datetime.combine(single_date, datetime.time(hour)).timestamp())

        for origin in origins:
            for destination in destinations:
                url = (
                    f"https://maps.googleapis.com/maps/api/distancematrix/json"
                    f"?origins={origin}&destinations={destination}"
                    f"&departure_time={departure_time}&key={API_KEY}"
                )
                try:
                    response = requests.get(url)
                    data = response.json()
                    
                    if data["status"] == "OK":
                        row = data["rows"][0]["elements"][0]
                        results.append({
                            "date": single_date.strftime("%Y-%m-%d"),
                            "hour": hour,
                            "origin": origin,
                            "destination": destination,
                            "distance_text": row.get("distance", {}).get("text"),
                            "distance_value": row.get("distance", {}).get("value"),
                            "duration_text": row.get("duration", {}).get("text"),
                            "duration_value": row.get("duration", {}).get("value"),
                            "duration_in_traffic_text": row.get("duration_in_traffic", {}).get("text"),
                            "duration_in_traffic_value": row.get("duration_in_traffic", {}).get("value")
                        })
                    else:
                        print(f"Failed for {origin} → {destination} at {hour}: {data['status']}")
                except Exception as e:
                    print(f"Error: {e}")
                
                # Respect API rate limits — wait a bit between calls
                time.sleep(1.1)

# Convert to DataFrame
df = pd.DataFrame(results)
display(df)

# Optional: save to DBFS or table
df.write.format("delta").save("/mnt/traffic_data/member_d_week1")

In [0]:
%sql select * from workspace.default.lahari

In [0]:
df = spark.sql("select distinct date from workspace.default.Traffic_analysis")

In [0]:
df.display()

In [0]:
%sql
SELECT * FROM workspace.default.samyuktha_trafficData

In [0]:
%sql
SELECT DISTINCT origin FROM workspace.default.traffic_analysis

In [0]:
# %sql
# SELECT date,date_format(date, 'EEEE') AS day_of_week
# FROM workspace.default.traffic_analysis

In [0]:
df_Sam = spark.sql("SELECT * FROM workspace.default.samyuktha_trafficData")

In [0]:
(df_Sam.count(), len(df_Sam.columns))

In [0]:
df_Sam.head()

In [0]:
df_Sam.write.insertInto("workspace.default.traffic_analysis", overwrite=False)

In [0]:
%sql
SELECT DISTINCT origin FROM workspace.default.traffic_analysis

Samyuktha

In [0]:
# Gyana inserting Data
df = spark.read.csv("C:/Users/hp/Downloads/traffic-data-project/traffic_data_week1.csv", header=True, inferSchema=True)
df.write.insertInto("workspace.default.traffic_analysis", overwrite=False)


Geyani

In [0]:
%sql 
describe table workspace.default.traffic_analysis

In [0]:
%sql
select distinct date, day_of_week from workspace.default.traffic_analysis limit 10

In [0]:
df = spark.table("workspace.default.traffic_analysis")
row_count = df.count()
row_count

In [0]:
%sql
select * from workspace.default.traffic_analysis limit 5

In [0]:
import holidays
from pyspark.sql.functions import udf, col
from pyspark.sql.types import BooleanType

table_name = "workspace.default.traffic_analysis"

df = spark.table(table_name)
india_holidays = holidays.India()

is_holiday_udf = udf(lambda d: d in india_holidays if d else False, BooleanType())
df_updated = df.withColumn("holiday", is_holiday_udf(col("date")))

df_updated.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(table_name)

In [0]:
from pyspark.sql.functions import when

# Update holiday column to true wherever day_of_week is Sunday
df_updated = df_updated.withColumn("holiday", when(col("day_of_week") == "Sunday", True).otherwise(col("holiday")))

# Save back to same table
df_updated.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(table_name)

In [0]:
%sql
select * from workspace.default.traffic_30 limit 5;